In [ ]:
# M0的折线图

In [1]:
import numpy as np
from tqdm import tqdm
import json

# 空间公共物品博弈类（保持原代码中的SPGG类不变）
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []
        self.avg_reputations = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S
    
    def fun_args_id(self, *args):
        return hash(args)
    
    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, *(group_offset)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, *(member_offset)) for s in result]
        self.cache[key] = result
        return result
    
    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result
    
    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        S1_of_member = S[1]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]
        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P
    
    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)
    
    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])
        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())
            
            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      P.sum(),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            avg_rep_coop_this_iter = np.mean(self.reputation[S_coop == 1]) if np.any(S_coop) else 0.0
            avg_rep_defect_this_iter = np.mean(self.reputation[S_def == 1]) if np.any(S_def) else 0.0
            self.avg_reputations.append((i, avg_rep_coop_this_iter, avg_rep_defect_this_iter))
            
            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break
            
            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))
                
                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])
                
                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)
                
                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()
        
        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L), np.sum(S_def) / (L * L),
                  P.sum(), np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total, final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

if __name__ == '__main__':
    # 参数设置
    r_values = np.arange(1.0, 5.1, 0.1)
    M0_values = np.arange(0.0, 0.202, 0.02)
    cost = 1
    L = 400
    iterations = 1000
    K = 0.1
    num_of_strategies = 2
    population_type = 0

    # 存储结果
    results = {M0: [] for M0 in M0_values}

    # 计算总任务数
    total_tasks = len(r_values) * len(M0_values)

    # 运行实验
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for M0 in M0_values:
            coop_ratios = []
            for r in r_values:
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=30, alpha=1.0, beta=1.0,
                            gamma=1.0, delta=1.0, reputation_threshold=15)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]
                coop_ratios.append(coop_ratio)
                pbar.update(1)
            results[M0] = coop_ratios

    # 保存结果到JSON文件
    output_data = {
        'r_values': r_values.tolist(),
        'M0_values': M0_values.tolist(),
        'results': {str(M0): ratios for M0, ratios in results.items()}
    }
    with open('M0_results.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 M0_results.json")

Running Experiments:   1%|▍                                                            | 3/451 [00:02<06:48,  1.10it/s]


KeyboardInterrupt: 

In [ ]:
# Rt的折线图

In [ ]:
import numpy as np
from tqdm import tqdm
import json

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []
        self.avg_reputations = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, *(group_offset)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, *(member_offset)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        S1_of_member = S[1]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])
        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      P.sum(),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            avg_rep_coop_this_iter = np.mean(self.reputation[S_coop == 1]) if np.any(S_coop) else 0.0
            avg_rep_defect_this_iter = np.mean(self.reputation[S_def == 1]) if np.any(S_def) else 0.0
            self.avg_reputations.append((i, avg_rep_coop_this_iter, avg_rep_defect_this_iter))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L), np.sum(S_def) / (L * L),
                  P.sum(), np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total, final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    # 参数设置
    r_values = np.arange(1.0, 5.1, 0.1)
    reputation_threshold_values = np.arange(0, 26, 5)
    M0 = 0.1
    cost = 1
    L = 400
    iterations = 1000
    K = 0.1
    num_of_strategies = 2
    population_type = 0

    # 存储结果
    results = {rt: [] for rt in reputation_threshold_values}
    total_tasks = len(r_values) * len(reputation_threshold_values)

    # 运行实验
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for r in r_values:
            for rt in reputation_threshold_values:
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=30, alpha=1.0, beta=1.0,
                            gamma=1.0, delta=1.0, reputation_threshold=rt)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]
                results[rt].append(coop_ratio)
                pbar.update(1)

    # 保存结果到JSON文件
    output_data = {
        'r_values': r_values.tolist(),
        'reputation_threshold_values': reputation_threshold_values.tolist(),
        'results': {str(rt): ratios for rt, ratios in results.items()}
    }
    with open('Rt_results.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 Rt_results.json")

Running Experiments:  57%|███████████████████████████████▌                       | 141/246 [1:56:36<2:04:59, 71.42s/it]

In [ ]:
# delta的折线图

In [ ]:
import numpy as np
from tqdm import tqdm
import json

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=1.0, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []
        self.avg_reputations = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, *(group_offset)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, *(member_offset)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        S1_of_member = S[1]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])
        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      P.sum(),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            avg_rep_coop_this_iter = np.mean(self.reputation[S_coop == 1]) if np.any(S_coop) else 0.0
            avg_rep_defect_this_iter = np.mean(self.reputation[S_def == 1]) if np.any(S_def) else 0.0
            self.avg_reputations.append((i, avg_rep_coop_this_iter, avg_rep_defect_this_iter))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L), np.sum(S_def) / (L * L),
                  P.sum(), np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total, final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    # 参数设置
    r_values = np.arange(1.0, 5.1, 0.1)
    beta_delta_values = np.arange(0.0, 1.1, 0.1)
    M0 = 0.1
    reputation_threshold = 15
    cost = 1
    L = 400
    iterations = 1000
    K = 0.1
    num_of_strategies = 2
    population_type = 0

    # 存储结果
    results = {bd: [] for bd in beta_delta_values}
    total_tasks = len(r_values) * len(beta_delta_values)

    # 运行实验
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for r in r_values:
            for bd in beta_delta_values:
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=30, alpha=1.0, beta=1.0, gamma=1.0,
                            delta=bd, reputation_threshold=reputation_threshold)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]
                results[bd].append(coop_ratio)
                pbar.update(1)

    # 保存结果到JSON文件
    output_data = {
        'r_values': r_values.tolist(),
        'delta_values': beta_delta_values.tolist(),
        'results': {str(bd): ratios for bd, ratios in results.items()}
    }
    with open('delta_results.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 delta_results.json")

In [ ]:
# 快照图、差值图、策略转换图和策略比例图

In [7]:
import numpy as np
import json

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []
        self.avg_reputations = []
        self.strategy_snapshots = []
        self.reputation_snapshots = []
        self.payoff_snapshots = []
        self.diff_snapshots = []
        self.payoff_strategy_diff_snapshots = []
        self.payoff_reputation_diff_snapshots = []
        self.transition_counts = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, *(group_offset)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, *(member_offset)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        S1_of_member = S[1]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])
        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        self.snapshots = []
        self.reputation_snapshots = []
        self.payoff_snapshots = []
        self.diff_snapshots = []
        self.payoff_strategy_diff_snapshots = []
        self.payoff_reputation_diff_snapshots = []
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_old = self._Sn.copy()
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      P.sum(),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            avg_rep_coop_this_iter = np.mean(self.reputation[S_coop == 1]) if np.any(S_coop) else 0.0
            avg_rep_defect_this_iter = np.mean(self.reputation[S_def == 1]) if np.any(S_def) else 0.0
            self.avg_reputations.append((i, avg_rep_coop_this_iter, avg_rep_defect_this_iter))

            if i in [1, 200, 800, self.iterations] or (i > self.iterations and np.sum(S_coop) == 0):
                self.strategy_snapshots.append((i, S_in_one.copy()))
                self.reputation_snapshots.append((i, self.reputation.copy()))
                self.payoff_snapshots.append((i, P.copy()))
                reputation_normalized = self.reputation / self.Rmax
                payoff_min = np.min(P)
                payoff_max = np.max(P)
                payoff_normalized = (P - payoff_min) / (payoff_max - payoff_min) if payoff_max != payoff_min else np.zeros_like(P)
                diff = S_in_one - reputation_normalized
                self.diff_snapshots.append((i, diff))
                payoff_strategy_diff = S_in_one - payoff_normalized
                self.payoff_strategy_diff_snapshots.append((i, payoff_strategy_diff))
                payoff_reputation_diff = payoff_normalized - reputation_normalized
                self.payoff_reputation_diff_snapshots.append((i, payoff_reputation_diff))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                S_new = S_in_one
                cc_count = np.sum((S_old == 0) & (S_new == 0))
                cd_count = np.sum((S_old == 0) & (S_new == 1))
                dc_count = np.sum((S_old == 1) & (S_new == 0))
                dd_count = np.sum((S_old == 1) & (S_new == 1))
                total_pop = L * L
                self.transition_counts.append((i, cc_count / total_pop, cd_count / total_pop, dc_count / total_pop, dd_count / total_pop))

                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L), np.sum(S_def) / (L * L),
                  P.sum(), np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total, final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    cost = 1
    records = {}
    spgg = SPGG(r=3.5, c=1, cost=cost, iterations=3000, L=100, num_of_strategies=2, K=0.1, population_type=0,
                Rmax=30, alpha=1.0, beta=1.0, gamma=1.0, delta=1.0, M0=0.02, reputation_threshold=1.5)
    final_record = spgg.run(log=False, func_on_iterate=None, records=records)
    print("最终记录:", final_record)

    # 保存数据到JSON文件
    output_data = {
        'parameters': {
            'r': spgg.r,
            'cost': spgg.cost,
            'iterations': spgg.iterations,
            'L': spgg.L,
            'K': spgg.K,
            'M0': spgg.M0,
            'reputation_threshold': spgg.reputation_threshold,
            'Rmax': spgg.Rmax,
            'alpha': spgg.alpha,
            'beta': spgg.beta,
            'gamma': spgg.gamma,
            'delta': spgg.delta
        },
        'ratios': spgg.ratios,
        'transition_counts': spgg.transition_counts,
        'strategy_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.strategy_snapshots],
        'reputation_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.reputation_snapshots],
        'payoff_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.payoff_snapshots],
        'diff_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.diff_snapshots],
        'payoff_strategy_diff_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.payoff_strategy_diff_snapshots],
        'payoff_reputation_diff_snapshots': [(iter_num, snapshot.tolist()) for iter_num, snapshot in spgg.payoff_reputation_diff_snapshots]
    }
    with open('experiment_snapshots.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 experiment_snapshots.json")

最终记录: (0.0, 1.0, -999.6000000000001, -0.09996000000000001, 0, -0.09996000000000001, 199.92000000000002, 0)
实验数据已保存到 experiment_snapshots.json


In [17]:
# 热力图（M0与Rt）

In [2]:
import numpy as np
import json
from tqdm import tqdm

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, group_offset, (0, 1)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, member_offset, (0, 1)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])

        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_old = self._Sn.copy()
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      np.sum(P),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                S_new = S_in_one
                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L),
                  np.sum(S_def) / (L * L),
                  np.sum(P),
                  np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total,
                  final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    # 参数设置
    M0_values = np.arange(0.0, 1.02, 0.02)  # M0 范围 0.0 到 1.0，步长 0.02
    reputation_thresholds = np.arange(0, 25.5, 0.5)  # Rt 范围 0 到 25，步长 0.5
    r = 3.5
    cost = 1
    L = 100
    iterations = 3000
    K = 0.1
    num_of_strategies = 2
    population_type = 0
    alpha = 1.0
    beta = 1.0
    gamma = 1.0
    delta = 1.0
    Rmax = 30

    # 存储结果
    results = {}
    Z = np.zeros((len(M0_values), len(reputation_thresholds)))

    # 运行实验，使用进度条
    total_tasks = len(M0_values) * len(reputation_thresholds)
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for i, M0 in enumerate(M0_values):
            for j, Rt in enumerate(reputation_thresholds):
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=Rmax, alpha=alpha, beta=beta,
                            gamma=gamma, delta=delta, reputation_threshold=Rt)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]  # 提取合作率
                Z[i, j] = coop_ratio
                pbar.update(1)
    results[r] = Z.tolist()  # 转换为列表以保存到JSON

    # 保存数据到JSON文件
    output_data = {
        'parameters': {
            'r': r,
            'cost': cost,
            'L': L,
            'iterations': iterations,
            'K': K,
            'num_of_strategies': num_of_strategies,
            'population_type': population_type,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'delta': delta,
            'Rmax': Rmax
        },
        'M0_values': M0_values.tolist(),
        'reputation_thresholds': reputation_thresholds.tolist(),
        'results': results
    }
    with open('experiment_results_MR.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 experiment_results_MR.json")

Running Experiments:   0%|                                                                    | 0/2601 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# 绘制热力图（M0与delta）

In [ ]:
import numpy as np
import json
from tqdm import tqdm

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, group_offset, (0, 1)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, member_offset, (0, 1)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])

        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      np.sum(P),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L),
                  np.sum(S_def) / (L * L),
                  np.sum(P),
                  np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total,
                  final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    # 参数设置
    M0_values = np.arange(0.0, 1.02, 0.02)  # M0 范围 0.0 到 1.0，步长 0.02
    delta_values = np.arange(0.0, 2.04, 0.04)  # delta 范围 0.0 到 2.0，步长 0.04
    r = 3.5
    cost = 1
    L = 100
    iterations = 3000
    K = 0.1
    num_of_strategies = 2
    population_type = 0
    alpha = 1.0
    beta = 1.0
    gamma = 1.0
    reputation_threshold = 15
    Rmax = 30

    # 存储结果
    results = {}
    Z = np.zeros((len(delta_values), len(M0_values)))  # 行为 Delta，列为 M0

    # 运行实验，使用进度条
    total_tasks = len(M0_values) * len(delta_values)
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for i, delta in enumerate(delta_values):
            for j, M0 in enumerate(M0_values):
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=Rmax, alpha=alpha, beta=beta,
                            gamma=gamma, delta=delta, reputation_threshold=reputation_threshold)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]  # 提取合作率
                Z[i, j] = coop_ratio
                pbar.update(1)
    results[r] = Z.tolist()  # 转换为列表以保存到JSON

    # 保存数据到JSON文件
    output_data = {
        'parameters': {
            'r': r,
            'cost': cost,
            'L': L,
            'iterations': iterations,
            'K': K,
            'num_of_strategies': num_of_strategies,
            'population_type': population_type,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'reputation_threshold': reputation_threshold,
            'Rmax': Rmax
        },
        'M0_values': M0_values.tolist(),
        'delta_values': delta_values.tolist(),
        'results': results
    }
    with open('experiment_results_MD.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 experiment_results_MD.json")

In [ ]:
# 绘制热力图（Rt与delta）

In [ ]:
import numpy as np
import json
from tqdm import tqdm

# 定义overlap5函数
overlap5 = lambda A: A + np.roll(A, -1, 0) + np.roll(A, 1, 0) + np.roll(A, -1, 1) + np.roll(A, 1, 1)

# 空间公共物品博弈类
class SPGG:
    def __init__(self, r=3, c=1, cost=0.5, K=0.1, L=50, iterations=1000, num_of_strategies=2, population_type=0, S_in_one=None,
                 Rmax=30, alpha=1.0, beta=0.01, gamma=1.0, delta=0.01, M0=0.0, reputation_threshold=15, **params):
        np.random.seed()
        all_params = dict(locals(), **params)
        del all_params['self'], all_params['params']
        self.params = all_params
        for key in self.params:
            setattr(self, key, self.params[key])
        self.cache = {}
        self._Sn = S_in_one
        self.create_population()
        self.it_records = []
        self.reputation = np.random.uniform(0, self.Rmax, size=(self.L, self.L))
        self.consecutive_coop = np.zeros((self.L, self.L))
        self.consecutive_defect = np.zeros((self.L, self.L))
        self.ratios = []

    def create_population(self):
        L = self.L
        S_in_one = self._Sn
        if S_in_one is None:
            S_in_one = np.random.randint(0, 2, size=L*L).reshape([L, L])
            self._Sn = S_in_one
        self._S = []
        for j in range(self.num_of_strategies):
            S = (S_in_one == j).astype(int)
            self._S.append(S)
        return self._S

    def fun_args_id(self, *args):
        return hash(args)

    def S(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("S", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        result = self._S
        if group_offset != (0, 0):
            result = [np.roll(s, group_offset, (0, 1)) for s in result]
        if member_offset != (0, 0):
            result = [np.roll(s, member_offset, (0, 1)) for s in result]
        self.cache[key] = result
        return result

    def N(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("N", group_offset)
        if key in self.cache:
            return self.cache[key]
        S = self.S(group_offset=group_offset)
        result = [overlap5(s) for s in S]
        self.cache[key] = result
        return result

    def P_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        key = self.fun_args_id("P_g_m", group_offset, member_offset)
        if key in self.cache:
            return self.cache[key]
        N = self.N(group_offset, member_offset)
        S = self.S(group_offset, member_offset)
        n = 5
        N0_in_group = N[0]
        S0_of_member = S[0]
        reputation = self.reputation
        reputation_threshold = self.reputation_threshold
        M0 = self.M0

        low_reputation = (reputation < reputation_threshold)
        N_low_rep = overlap5(low_reputation.astype(float))
        M0_contribution = M0 * N_low_rep
        coop_contribution = self.c * N0_in_group
        c_pool = self.r * coop_contribution / n
        M0_pool = np.zeros_like(S0_of_member, dtype=float)
        mask_coop_in_group = (N0_in_group > 0)
        M0_pool[mask_coop_in_group] = (self.r * M0_contribution[mask_coop_in_group]) / N0_in_group[mask_coop_in_group]

        P = np.zeros_like(S0_of_member, dtype=float)
        P += c_pool
        P += np.where(S0_of_member == 1, M0_pool, 0)
        P -= np.where(low_reputation, M0, 0)
        P -= self.cost * S0_of_member

        if np.any(np.isnan(P)):
            print("P_g_m payoff calculation contains nan")
        self.cache[key] = P
        return P

    def P_AT_g_m(self, group_offset=(0, 0), member_offset=(0, 0)):
        return self.P_g_m(group_offset, member_offset)

    def update_reputation(self):
        S_list_current = self._S
        S0, S1 = S_list_current[0], S_list_current[1]
        Rt_old = self.reputation.copy()
        C_old = self.consecutive_coop.copy()
        D_old = self.consecutive_defect.copy()

        C_new = np.where(S0 == 1, C_old + 1, 0)
        D_new = np.where(S1 == 1, D_old + 1, 0)
        C_new = np.minimum(C_new, 100)
        D_new = np.minimum(D_new, 100)

        reputation_increase = np.zeros_like(Rt_old)
        mask_coop = (S0 == 1)
        if np.any(mask_coop):
            reputation_increase[mask_coop] = self.alpha * np.log(np.exp(1) + self.beta * C_new[mask_coop])

        reputation_decrease = np.zeros_like(Rt_old)
        mask_defect = (S1 == 1)
        if np.any(mask_defect):
            reputation_decrease[mask_defect] = self.gamma * np.exp(self.delta * D_new[mask_defect])

        if np.any(np.isnan(reputation_increase[mask_coop])) and np.any(mask_coop):
            print("reputation_increase (for cooperators) contains nan")
        if np.any(np.isnan(reputation_decrease[mask_defect])) and np.any(mask_defect):
            print("reputation_decrease (for defectors) contains nan")

        Rt_new = Rt_old + reputation_increase - reputation_decrease
        Rt_new = np.minimum(Rt_new, self.Rmax)
        Rt_new = np.maximum(Rt_new, 0)

        if np.any(np.isnan(Rt_new)):
            print("Reputation contains nan after update")

        self.reputation = Rt_new
        self.consecutive_coop = C_new
        self.consecutive_defect = D_new

    def run(self, log=False, records={}, func_on_iterate=None, update=True):
        L, K = self.L, self.K
        for i in range(1, self.iterations + 1):
            self.cache = {}
            S_in_one = self._Sn
            n = 5
            P = self.P_AT_g_m() + \
                self.P_AT_g_m((1, 0), (-1, 0)) + \
                self.P_AT_g_m((-1, 0), (1, 0)) + \
                self.P_AT_g_m((1, 1), (-1, 1)) + \
                self.P_AT_g_m((-1, 1), (1, 1))
            self.P = P

            if func_on_iterate:
                func_on_iterate(locals())

            S = self.S()
            S_coop, S_def = S[0], S[1]
            record = (np.sum(S_coop) / (L * L),
                      np.sum(S_def) / (L * L),
                      np.sum(P),
                      np.average(P),
                      np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                      np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0)
            self.it_records.append(record)

            coop_ratio_this_iter = np.sum(S_coop) / (L * L)
            defect_ratio_this_iter = np.sum(S_def) / (L * L)
            self.ratios.append((i, coop_ratio_this_iter, defect_ratio_this_iter))

            if np.sum(S_coop) == 0:
                break
            if i == self.iterations:
                break

            if update:
                W_w = 1 / (1 + np.exp((P - np.roll(P, 1, axis=1)) / K))
                W_e = 1 / (1 + np.exp((P - np.roll(P, -1, axis=1)) / K))
                W_n = 1 / (1 + np.exp((P - np.roll(P, 1, axis=0)) / K))
                W_s = 1 / (1 + np.exp((P - np.roll(P, -1, axis=0)) / K))

                RandomNeighbour = np.random.randint(0, 4, size=L * L).reshape([L, L])
                Random01 = np.random.uniform(0, 1, size=L * L).reshape([L, L])

                S_in_one = (RandomNeighbour == 0) * ((Random01 <= W_w) * np.roll(S_in_one, 1, axis=1) + (Random01 > W_w) * S_in_one) + \
                           (RandomNeighbour == 1) * ((Random01 <= W_e) * np.roll(S_in_one, -1, axis=1) + (Random01 > W_e) * S_in_one) + \
                           (RandomNeighbour == 2) * ((Random01 <= W_n) * np.roll(S_in_one, 1, axis=0) + (Random01 > W_n) * S_in_one) + \
                           (RandomNeighbour == 3) * ((Random01 <= W_s) * np.roll(S_in_one, -1, axis=0) + (Random01 > W_s) * S_in_one)

                self._Sn = S_in_one
                self._S = []
                for j in range(self.num_of_strategies):
                    S = (S_in_one == j).astype(int)
                    self._S.append(S)

                self.update_reputation()

        S = self.S()
        S_coop, S_def = S[0], S[1]
        P = self.P_AT_g_m() + \
            self.P_AT_g_m((1, 0), (-1, 0)) + \
            self.P_AT_g_m((-1, 0), (1, 0)) + \
            self.P_AT_g_m((1, 1), (-1, 1)) + \
            self.P_AT_g_m((-1, 1), (1, 1))
        low_reputation = (self.reputation < self.reputation_threshold)
        final_deposit_deducted_total = np.sum(np.where(low_reputation, self.M0, 0))
        final_deposit_refunded_total = 0
        record = (np.sum(S_coop) / (L * L),
                  np.sum(S_def) / (L * L),
                  np.sum(P),
                  np.average(P),
                  np.average(P[S_in_one == 0]) if np.sum(S_coop) > 0 else 0,
                  np.average(P[S_in_one == 1]) if np.sum(S_def) > 0 else 0,
                  final_deposit_deducted_total,
                  final_deposit_refunded_total)
        records[(self.r, self.cost)] = record
        return record

if __name__ == '__main__':
    # 参数设置
    reputation_thresholds = np.arange(0, 25.5, 0.5)  # Rt 范围 0 到 25，步长 0.5
    delta_values = np.arange(0.0, 2.04, 0.04)  # delta 范围 0.0 到 2.0，步长 0.04
    r = 3.5
    cost = 1
    L = 100
    iterations = 3000
    K = 0.1
    num_of_strategies = 2
    population_type = 0
    alpha = 1.0
    beta = 1.0
    gamma = 1.0
    M0 = 0.2
    Rmax = 30

    # 存储结果
    results = {}
    Z = np.zeros((len(reputation_thresholds), len(delta_values)))  # 行为 Rt，列为 delta

    # 运行实验，使用进度条
    total_tasks = len(reputation_thresholds) * len(delta_values)
    with tqdm(total=total_tasks, desc="Running Experiments") as pbar:
        for i, Rt in enumerate(reputation_thresholds):
            for j, delta in enumerate(delta_values):
                records = {}
                spgg = SPGG(r=r, c=1, cost=cost, iterations=iterations, L=L, num_of_strategies=num_of_strategies,
                            K=K, population_type=population_type, M0=M0, Rmax=Rmax, alpha=alpha, beta=beta,
                            gamma=gamma, delta=delta, reputation_threshold=Rt)
                final_record = spgg.run(log=False, records=records)
                coop_ratio = final_record[0]  # 提取合作率
                Z[i, j] = coop_ratio
                pbar.update(1)
    results[r] = Z.tolist()  # 转换为列表以保存到JSON

    # 保存数据到JSON文件
    output_data = {
        'parameters': {
            'r': r,
            'cost': cost,
            'L': L,
            'iterations': iterations,
            'K': K,
            'num_of_strategies': num_of_strategies,
            'population_type': population_type,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'M0': M0,
            'Rmax': Rmax
        },
        'reputation_thresholds': reputation_thresholds.tolist(),
        'delta_values': delta_values.tolist(),
        'results': results
    }
    with open('experiment_results_RD.json', 'w') as f:
        json.dump(output_data, f, indent=4)
    print("实验数据已保存到 experiment_results_RD.json")